In [1]:
"""
use the HOG descriptors to augment the data set and then put that through a decision tree, random forest, AdaBoost, and XGBoost
"""

'\nuse the HOG descriptors to augment the data set and then put that through a decision tree, random forest, AdaBoost, and XGBoost\n'

In [2]:
"""
Run once ever

Downloads and extracts the dataset
"""


# import urllib.request
# import tarfile
# import os

# mp3file = urllib.request.urlopen("https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz")
# with open('tmp','wb') as output:
#   output.write(mp3file.read())

# file = tarfile.open("tmp")
# file.extractall(".")
# file.close()
# os.remove("tmp")
# os.remove("cifar-100-python/file.txt~")

'\nRun once ever\n\nDownloads and extracts the dataset\n'

In [3]:

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import cv2
from collections import Counter
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#function to read files present in the Python version of the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [5]:
trainData = unpickle('./cifar-100-python/train')
#type of items in each file
for item in trainData:
    print(item, type(trainData[item]))

filenames <class 'list'>
batch_label <class 'str'>
fine_labels <class 'list'>
coarse_labels <class 'list'>
data <class 'numpy.ndarray'>


In [6]:
testData = unpickle('./cifar-100-python//test')
metaData = unpickle('./cifar-100-python//meta')
#metaData
print("Fine labels:", metaData['fine_label_names'], "\n")
print("Coarse labels:", metaData['coarse_label_names'])

Fine labels: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', '

In [7]:
#storing fine labels along with its number code in a dataframe
subCategory = pd.DataFrame(metaData['fine_label_names'], columns=['SubClass'])
print(subCategory)

         SubClass
0           apple
1   aquarium_fish
2            baby
3            bear
4          beaver
..            ...
95          whale
96    willow_tree
97           wolf
98          woman
99           worm

[100 rows x 1 columns]


In [8]:
X_train = trainData['data']
X_train = X_train.reshape(len(X_train),3,32,32).transpose(0,2,3,1)

In [9]:
X_test = testData['data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
#X_test

In [10]:
winSize = (32,32)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (4,4)
nbins = 9


df = []

In [11]:

df = []
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
for i in range(len(X_train)):
    image = cv2.cvtColor(X_train[i],cv2.COLOR_BGR2GRAY)
    df.append(hog.compute(image))
    
DF_train = pd.DataFrame(df)

df = []
for i in range(len(X_test)):
    image = cv2.cvtColor(X_test[i],cv2.COLOR_BGR2GRAY)
    df.append(hog.compute(image))

DF_test = pd.DataFrame(df)


In [12]:
print(DF_train)

           0         1         2         3         4         5         6     \
0      0.032799  0.015301  0.045787  0.000000  0.069334  0.000000  0.000000   
1      0.006916  0.000000  0.000000  0.000000  0.241620  0.000000  0.000000   
2      0.327324  0.175260  0.089584  0.062461  0.073516  0.000000  0.000000   
3      0.097244  0.235721  0.044991  0.010438  0.005530  0.000000  0.000000   
4      0.227998  0.040255  0.278890  0.130381  0.014747  0.007472  0.119501   
...         ...       ...       ...       ...       ...       ...       ...   
49995  0.014346  0.002504  0.039960  0.024616  0.042274  0.044265  0.236928   
49996  0.049246  0.108321  0.099552  0.040950  0.175260  0.170823  0.130690   
49997  0.105599  0.160996  0.087342  0.291104  0.379031  0.055069  0.008135   
49998  0.052827  0.020290  0.005121  0.309802  0.309802  0.178927  0.063858   
49999  0.316890  0.181016  0.038011  0.000000  0.007353  0.000000  0.000000   

           7         8         9     ...      1754 

In [ ]:
"""
Augment the training data with these HOG descriptors
"""

trainData = unpickle('./cifar-100-python/train')
testData = unpickle('./cifar-100-python//test')

trainData.pop("batch_label")
trainData.pop("filenames")
d = trainData.pop("data")

data = pd.DataFrame.from_dict(trainData)
d = pd.DataFrame(d)
data = pd.concat([data, d], axis=1)
X_train = data[(x for x in range(3072))]
y_train = data["fine_labels"]

testData.pop("batch_label")
testData.pop("filenames")
d = testData.pop("data")

data = pd.DataFrame.from_dict(testData)
d = pd.DataFrame(d)
data = pd.concat([data, d], axis=1)

X_test = data[(x for x in range(3072))]
y_test = data["fine_labels"]

In [14]:

#augmentedData = [...augmentedData, ...DF];
# print(augmentedData)
# print(X_train)
# X_train = X_train[0:5000]
# y_train = y_train[0:5000]
X_train = pd.concat([X_train, DF_train], axis=1)
X_test = pd.concat([X_test, DF_test], axis=1)

In [15]:

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import r2_score, mean_squared_error, zero_one_loss
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import threading

In [16]:

# tree = DecisionTreeClassifier()
# _ = tree.fit(augmentedData, y_train)
# print(tree.get_depth())
# print(tree.score(X_test, y_test))

In [ ]:

"""
With the limited dataset of HOG descriptors (5000 examples):
    Best depth was 31, 32, and 35, each achieving a 5.4% accuracy
    This is worse than without the HOG descriptors
With full dataset for HOG descriptors:
    152 max depth and 8.7% accuracy
    Best depth was 20, achieving 9.37% accuracy
    No better than without HOG descriptors
"""
# def train(depth: int):
#     tree = DecisionTreeClassifier(max_depth=depth)
#     _ = tree.fit(X_train, y_train)
#     print(f"Depth: {depth}, \tAccuracy: {tree.score(X_test, y_test)}")

# threads = []
# # uncomment below to train the basic decision tree
# for i in range(152):
#     threads.append(threading.Thread(target=train, args=(i,)))
#     threads[len(threads) - 1].start()
#     # limit to 20 threads being used
#     # adjust to fit your computer
#     if len(threads) > 20:
#         threads[0].join()
#         threads.pop(0)
# for thread in threads:
#     thread.join()

In [ ]:
"""
AdaBoost

Did nothing useful
7.75% accuracy
"""

# adaBoost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=1000)
# adaBoost = AdaBoostClassifier()
# adaBoost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
# y_pred = adaBoost.predict(X_test);
# print(1 - zero_one_loss(y_test, y_pred))

0.021499999999999964


In [ ]:
"""
XGBoost

Did nothing useful
1% accuracy
"""

# xgBoost = GradientBoostingRegressor(learning_rate=0.01, n_estimators=100);
# xgBoost = GradientBoostingRegressor()
# xgBoost.fit(X_train, y_train)

TypeError: GradientBoostingRegressor.__init__() got an unexpected keyword argument 'estimator'

In [ ]:
# print(xgBoost.score(X_test, y_test))
# y_pred = xgBoost.predict(X_test).round()
# print(y_pred)
# print(1 - zero_one_loss(y_test, y_pred))

0.010099999999999998
